# Detección de diabetes

## Objetivos: 

* Construir modelos que puedan **clasificar** una persona en diabética o no a partir de información medica.


* Encontrar aquellas **variables** son **más relevantes** para la clasificación.


* Discutir cuál es una **buena métrica** para evaluar el desempeño de los modelos.

### Sobre el dataset:
Proviene del **"National Institute of Diabetes and Digestive and Kidney Diseases"**

Este dataset es un subconjunto de un dataset mucho mayor con las restricciones:

* **Mujeres** 

* **Mayores de 21 años** 

* **De ascendencia Pima India**

In [1]:
library(ggplot2)
library(GGally)
library(reshape2)
library(klaR)

Warning message:
"package 'ggplot2' was built under R version 3.6.3"Warning message:
"package 'GGally' was built under R version 3.6.3"Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2
Warning message:
"package 'klaR' was built under R version 3.6.3"Loading required package: MASS


In [2]:
datos <- read.csv("diabetes.csv")

In [3]:
fig <- function(width, heigth){
     options(repr.plot.width = width, repr.plot.height = heigth)
}

## Primeras entradas del dataset:

In [4]:
head(datos)

Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
6,148,72,35,0,33.6,0.627,50,1
1,85,66,29,0,26.6,0.351,31,0
8,183,64,0,0,23.3,0.672,32,1
1,89,66,23,94,28.1,0.167,21,0
0,137,40,35,168,43.1,2.288,33,1
5,116,74,0,0,25.6,0.201,30,0


* **DiabetesPedigreeFunction:** score que es mayor mientras mayor sea la probabilidad de tener diabetes basado en historia familiar

* **Outcome:** 0 si la persona no es diabética, 1 si lo es

## Análisis exploratorio

### Sobre el dataset

In [5]:
table(datos$Outcome)


  0   1 
500 268 

El dataset contiene información acerca de 768 mujeres, **500 de las cuales no son diabéticas y 268 si** lo son, por lo que las clases en el dataset no están balanceadas.

### Boxplots

In [6]:
# Para cada una de las variables se realizo un boxplot de la siguiente manera:
# ggplot(datos, aes(x=factor(Outcome), y=DiabetesPedigreeFunction, fill=factor(Outcome))) + geom_boxplot()

<img src="Boxplots.png" width="800">

#### Observaciones de los boxplot:
* **1** - La cantidad de **glucosa** en sangre es mayor en las personas con diabetes. El 75 % de las mujeres sin diabetes tiene un nivel de glucosa menor a 125 mientras que un 75 % de las mujeres con diabetes tiene un nivel de glucosa mayor a 120.


* **2** - La mediana de la **insulina** en sangre para las personas con diabetes es aproximadamente 0. Sin embargo hay personas diabéticas que tienen una cantidad de insulina relativamentenormal o incluso elevada. 


* **3** - La **Función de Pedigree de Diabetes** no presenta muchas diferencias para personas diabéticay no diabéticas. Es levemente mayor en el caso de las personas con diabetes lo cual esesperado. Mujeres con **historial familiar similar.**


* **4** - **Cantidad de embarazos, edad y BMI** aumentan para personas diabéticas. **Presión sanguínea y grosor de piel** también pero más levemente.

### Correlaciones

In [7]:
round(cor(datos),2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
Pregnancies,1.00,0.13,0.14,-0.08,-0.07,0.02,-0.03,0.54,0.22
Glucose,0.13,1.00,0.15,0.06,0.33,0.22,0.14,0.26,0.47
BloodPressure,0.14,0.15,1.00,0.21,0.09,0.28,0.04,0.24,0.07
SkinThickness,-0.08,0.06,0.21,1.00,0.44,0.39,0.18,-0.11,0.07
Insulin,-0.07,0.33,0.09,0.44,1.00,0.20,0.19,-0.04,0.13
BMI,0.02,0.22,0.28,0.39,0.20,1.00,0.14,0.04,0.29
DiabetesPedigreeFunction,-0.03,0.14,0.04,0.18,0.19,0.14,1.00,0.03,0.17
Age,0.54,0.26,0.24,-0.11,-0.04,0.04,0.03,1.00,0.24
Outcome,0.22,0.47,0.07,0.07,0.13,0.29,0.17,0.24,1.00


#### Observaciones en las correlaciones:

* **1** - Las variables **no están muy correlacionadas** entre si, en general las correlaciones son menores a en modulo 0.3


* **2** - Existe una correlación positiva entre **edad y cantidad de embarazos.**


* **3** - **Insulina y glucosa** están correlacionados positivamente.


* **4** - La variable mas correlacionada con el resultado o outcome es la **glucosa** en sangre y las iguiente el **BMI**

# LDA/QDA

En primer lugar se realizaran LDA y QDA unicamente para las variables Glucosa y BMI con el objetivo de comparar la particion del espacio de decision con los del arbol y del bootstrap.

#### Nota: Antes de realizar un analisis de LDA y QDA siempre se debe testear normalidad, es un supuesto fuerte del modelo

Busco las fronteras de division entre las personas dibieticas y no diabeticas segun QDA. Para ello hay que buscar estimaciones de:

$\hat\Sigma_1$, $\hat\Sigma_2$ y $\hat\Sigma$ las matrices de covarianza de las personas diabeticas, no diabeticas y un promedio ponderado de ambas.

$\hat\mu_1$ y $\hat\mu_2$ El vector de medias [Glucosa, BMI] para personas diabeticas y no diabeticas
 
$\hatπ_1$ y $\hatπ_2$ Estimacion de la probabilidad a priori de pertenecer a cada clase.

In [8]:
group1 = datos[datos$Outcome==0,]
group2 = datos[datos$Outcome==1,]

# En la columna 6 esta BMI y en la 2 Glucosa
cov1 = cov(group1[,c(6,2)])
cov2 = cov(group2[,c(6,2)])

n1 = nrow(group1)
n2 = nrow(group2)

n = n1+n2
K = 2

Sigma <- 1/(n - K) * (cov1*(n1-1) + cov2*(n2-1))
SigmaInv = solve(Sigma);

In [9]:
cov1
cov2
Sigma

,BMI,Glucose
BMI,59.13387,26.48445
Glucose,26.48445,683.36232


,BMI,Glucose
BMI,52.75069,11.69575
Glucose,11.69575,1020.13946


,BMI,Glucose
BMI,56.90892,21.32964
Glucose,21.32964,800.75070


In [10]:
mu_1 = rbind(mean(group1$BMI),mean(group1$Glucose))
mu_2 = rbind(mean(group2$BMI),mean(group2$Glucose))
mu_1
mu_2

30.3042
109.9800


35.14254
141.25746


### Los resultados obtenidos fueron los siguientes:

$\hat\Sigma_1$ = $\begin{bmatrix}
59.133	& 26.484 \\
26.484	& 683.362
\end{bmatrix}$


$\hat\Sigma_2 = \begin{bmatrix}
52.750 & 11.695 \\
11.695 & 1020.139
\end{bmatrix}$


$\hat\Sigma = \begin{bmatrix}
56.908 & 21.329 \\
21.329 & 800.750
\end{bmatrix}$

$\hat\mu_1 = \begin{bmatrix}
30.304 \\
109.980 
\end{bmatrix}$

$\hat\mu_2 = \begin{bmatrix}
35.142 \\
141.257
\end{bmatrix}$
 
$\hatπ_1= \frac{500}{768}$ 

$\hatπ_2= \frac{268}{768}$

Para hallar las fronteras de decision se debe calcular:

$L_1(x) - L_2(X)$ donde $L_i(x)= \mu_i^T \Sigma^{-1} (x-\frac{1}{2}\mu_i)+\text{log}\pi_i$ 

$Q_1(x) - Q_2(x)$ donde $Q_i(x) = -\frac{1}{2}\text{log}|\Sigma_i|-\frac{1}{2}(x-\mu_{i})^{T}\Sigma_{i}^{-1}(x-\mu_{i})+\text{log}\pi_i$

In [11]:
#Realizando las cuentas: 
# L1 - L2 = −0.0371y−0.07x+ 7.623 = 0
# Q1 - Q2 = 1.024x^2+ 0.408xy−0.215y^2−200.5x−1,07y=−7197,01

Estas funciones dividen al espacio de decision de la siguiente manera:

In [12]:
#partimat(as.factor(datos$Outcome) ~datos$Glucose + datos$BMI, method="lda", col.correct=NA, col.wrong=NA)

<img src="ldapartition.png" width="500" height="500"> 

In [13]:
#partimat(as.factor(datos$Outcome) ~datos$Glucose + datos$BMI, method="qda", col.correct=NA, col.wrong=NA, precision =100)

<img src="qdapartition.png" width="500" height="500"> 

### Observaciones:

* QDA tiene mayor flexibilidad en la frontera de las regiones, esto sucede porque el algoritmo realiza menos supuestos acerca de los datos, en particular no presupone que la matriz de covarianza es la misma para todas las clases.


* Las regiones halladas con QDA son mas similares a las halladas con el árbol de decisión y su desempeño también.


* Las regiones halladas con LDA son mas similares a las halladas con KNN y su desempeño también.

### Predicciones con QDA - Glucosa y BMI

In [14]:
clases_predichas_qda <- qda(datos$Outcome ~ datos$BMI+ datos$Glucose, data=datos, na.action="na.omit", CV=TRUE) #CV = TRUE utiliza jack-knifed leave-one out CV

In [15]:
ct <- table(datos$Outcome, clases_predichas_qda$class)
ct

   
      0   1
  0 449  51
  1 133 135

In [16]:
accuracy = (449 + 135) / (449 + 135 + 51 + 133)
precision = 135/ (135+51)
recall = 135/(135+133)

In [17]:
round(accuracy,3)
round(precision,3)
round(recall,3)

[1] 0.76

[1] 0.726

[1] 0.504

### Predicciones con LDA - Glucosa y BMI

In [18]:
clases_predichas_lda <- lda(datos$Outcome ~ datos$BMI+ datos$Glucose, data=datos, na.action="na.omit", CV=TRUE) #CV = TRUE utiliza jack-knifed leave-one out CV

In [19]:
ct <- table(datos$Outcome, clases_predichas_lda$class)
ct

   
      0   1
  0 448  52
  1 125 143

In [20]:
accuracy = (448 + 143) / (448 + 143 + 52 + 125)
precision = 143/ (143+52)
recall = 143/(143+125)

In [21]:
round(accuracy,3)
round(precision,3)
round(recall,3)

[1] 0.77

[1] 0.733

[1] 0.534

### Predicciones con LDA - Todas las variables

In [22]:
clases_predichas_lda <- lda(datos$Outcome ~ datos$BMI+ datos$Glucose + datos$Insulin+ datos$Age +  datos$Pregnancies+ datos$SkinThickness +  datos$BloodPressure+ datos$DiabetesPedigreeFunction, data=datos, na.action="na.omit", CV=TRUE) #CV = TRUE utiliza jack-knifed leave-one out CV

In [23]:
ct <- table(datos$Outcome, clases_predichas_lda$class)
ct

   
      0   1
  0 442  58
  1 115 153

In [24]:
accuracy = (442 + 153) / (442 + 153 + 58 + 115)
precision = 153/ (153+58)
recall = 153/(153+115)

In [25]:
round(accuracy,3)
round(precision,3)
round(recall,3)

[1] 0.775

[1] 0.725

[1] 0.571

### Predicciones con QDA - Todas las variables

In [26]:
clases_predichas_qda <- qda(datos$Outcome ~ datos$BMI+ datos$Glucose + datos$Insulin+ datos$Age +  datos$Pregnancies+ datos$SkinThickness +  datos$BloodPressure+ datos$DiabetesPedigreeFunction, data=datos, na.action="na.omit", CV=TRUE) #CV = TRUE utiliza jack-knifed leave-one out CV

In [27]:
ct <- table(datos$Outcome, clases_predichas_qda$class)
ct

   
      0   1
  0 420  80
  1 120 148

In [28]:
accuracy = (420 + 148) / (420 + 148 + 80 + 120)
precision = 148/ (148+80)
recall = 148/(148+120)

In [29]:
round(accuracy,3)
round(precision,3)
round(recall,3)

[1] 0.74

[1] 0.649

[1] 0.552

### Resultados de LDA/QDA

<img src="resultadosldaqda.png" width="400" height="800">

## Resumen de resultados LDA/QDA

### Conclusiones

#### Se obtuvieron las siguientes conclusiones a partir de la realización del trabajo practico:


* Las **variables mas relevantes para realizar la clasificación son Glucosa y BMI**, pues:
     * Son las variables con mayor correlación con el resultado que indica si una persona esdiabética o no.
     * Los boxplots muestran que los valores de estas variables son significativamente diferentespara las personas diabéticas que para las personas no diabéticas
     * El scatter plot de BMI vs. Glucosa muestra que estas variables separan bien a lasobservaciones de cada clase.



* Al utilizar los distintos modelos con estas variables se obtuvieron resultados similares que al utilizar los modelos con todas las variables.
  
  
* En todos los modelos el **recall fue la métrica con menor valor**. Esta es una métrica muy importante sobre todo en este caso, el siguiente ejemplo lo demuestra. Si a las personasque se predice con diabetes reciben un medicamento y a las personas que se predicen nodiabéticas no lo reciben, pueden darse las siguientes situaciones:
    
     * Una persona con diabetes recibe el medicamento
     * Una persona sin diabetes recibe el medicamento
     * Una persona **con diabetes no recibe el medicamento**
     * Una persona sin diabetes no recibe el medicamento
  
  
Claramente el error que se comete cuando una persona con diabetes no recibe el medicamento que le es necesario es peor a el que se comete cuando una persona sin diabetes lo recibe. Es decir, hay que buscar tener pocos falsos negativos (personas que se clasifican no diabéticas y que en verdad lo son). Al final de las conclusiones se proponen algunos cambios posibles alos algoritmos hallados para intentar mejorar esta métrica.


* La división del espacio de decisión formado por las variables glucosa y BMI es muy diferentes egún el algoritmo que se utilice.

### Mejoras propuestas

* Una potencial mejora para este proyecto podría realizarse obteniendo un dataset menossesgado. El dataset utilizado en el trabajo tiene información solo de mujeres y que además comparten ascendencia, lo cual potencialmente reduce la extrapolación de estos resultados.


* De todos los algoritmos utilizados en el trabajo puede extraerse una estimación de probabi-lidad de pertenecer a una clase. Por ejemplo:

    •En el árbol de decisión: la cantidad de observaciones de cada clase que hay en las hojasdel árbol pueden utilizarse como estimación para la probabilidad de que una observaciónpertenezca a una clase.
    
    •En bagging: mientras mas arboles predigan que una observación es de cierta clase,mayor sera la probabilidad de que sea de esa clase.
    
    •En KNN: contando la cantidad de vecinos de una clase se puede obtener la probabilidadde que sea de esa clase
    
Seria interesante ver que ocurre con el recall y el precision al variar el umbral de probabilidad para el cual uno considera que una persona es diabática.